In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn import metrics
from ML.module import ClassificationModel, RegressionModel
sys.path.append("./ML")

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [2]:
external_data_root = Path("./CGCNN_MT/data/")
ts_external_csv = external_data_root/"TS_external_test/RAC_and_zeo_features_with_id_prop.csv"
ws_external_csv = external_data_root/"WS24v2_external_test/RAC_and_zeo_features_with_id_prop.csv"
log_dir = Path("./CGCNN_MT/evaluation")

df_ts = pd.read_csv(ts_external_csv)
df_ws = pd.read_csv(ws_external_csv)

print(df_ts.shape, df_ws.shape)

(40, 195) (46, 197)


In [3]:
df_ts.head()

,MofName,Partition,ts_label,ts2_label,ss_label,Di,Df,Dif,rho,VSA,...,f-lig-T-2,f-lig-T-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3
0,AVAQIX_clean,external_test,370.271072,1,1,5.44587,3.64311,5.42872,1.30707,0.000,...,190.0,168.0,397.0,768.0,1020.0,884.0,91.2838,176.0010,261.8652,273.5720
1,AVIHIY_clean,external_test,649.652778,1,1,6.88379,6.45442,6.88379,1.62124,1386.590,...,294.0,306.0,711.0,1260.0,1896.0,1944.0,144.0441,255.9690,399.6456,462.2130
2,AVIHOE_clean,external_test,642.086331,1,1,6.82367,6.45553,6.82367,1.66276,1367.050,...,294.0,306.0,711.0,1260.0,1896.0,1944.0,144.0441,255.9690,399.6456,462.2130
3,BIBBUL_clean,external_test,269.374455,0,1,3.85878,3.31702,3.85878,1.32039,0.000,...,542.0,575.0,719.0,1572.0,2226.0,2165.0,187.5020,395.3010,652.1304,750.9986
4,CODNAM_clean,external_test,328.079888,0,0,4.58434,3.97684,4.58339,1.63521,947.884,...,12.0,0.0,418.0,750.0,432.0,0.0,45.1369,54.8376,116.4096,0.0000


## Get best ML models

In [4]:
model_root_dir = "./ML/models"
model_root_dir = Path(model_root_dir)
cls_tasks = ["SSD", "WS24"]
reg_tasks = ["TSD"]
all_metrics = {}
for task_set, tp in zip([cls_tasks, reg_tasks], ["cls", "reg"]):
    dfs = []
    for task in task_set:
        model_dir = model_root_dir / task/ "RAC_and_zeo_features_with_id_prop"
        for subdir in model_dir.iterdir():
            if not subdir.is_dir():
                continue
            model_metrics_file = subdir / "model_metrics.csv"
            df_metrics = pd.read_csv(model_metrics_file, index_col=0)
            keep_cols = [col for col in df_metrics.columns if "tr_" not in col and "val_" not in col]
            df_metrics = df_metrics.loc[["mean"], keep_cols].copy()
            df_metrics.insert(0, "Task", "_".join([task, subdir.name]).replace("_label", "").replace("_Label", ""))
            dfs.append(df_metrics)
            if tp == "cls":
                df_metrics.sort_values(by=["te_AUC"], inplace=True, ascending=False)
            else:
                df_metrics.sort_values(by=["te_MAE"], inplace=True)
            df_metrics.columns = [col.replace("te_", "") for col in df_metrics.columns]
            all_metrics[task + "_" + subdir.name] = df_metrics.to_dict("records")[0]
df_ml_our = pd.DataFrame(all_metrics).T.copy()
df_ml_our

,Task,ACC,BACC,MCC,AUC,F1,CE,model_name,feature_selector,feature_num,random_state,hyperparams,R2,RMSE,MAE,Pearson,Spearman
SSD_Label,SSD,0.7615,0.7261,0.4917,0.8342,0.734,0.510013623930981,RandomForestClassifier,mutual_info,80,0,NaN,NaN,NaN,NaN,NaN,NaN
WS24_water4_label,WS24_water4,0.6393,0.5154,0.3854,0.8479,0.5313,1.06965887257461,RandomForestClassifier,mutual_info,171,0,NaN,NaN,NaN,NaN,NaN,NaN
WS24_boiling_label,WS24_boiling,0.7308,0.7308,0.4743,0.6805,0.7271,0.6441295926387494,RandomForestClassifier,mutual_info,5,0,NaN,NaN,NaN,NaN,NaN,NaN
WS24_acid_label,WS24_acid,0.8056,0.8056,0.6198,0.9198,0.8042,0.48261490733259255,GaussianProcessClassifier,RFE,6,0,NaN,NaN,NaN,NaN,NaN,NaN
WS24_water_label,WS24_water,0.7123,0.6868,0.3886,0.7697,0.6906,0.5860507109901745,RandomForestClassifier,RFE,85,0,NaN,NaN,NaN,NaN,NaN,NaN
WS24_base_label,WS24_base,0.7083,0.7083,0.4181,0.7431,0.7078,0.6270048109193643,GaussianProcessClassifier,mutual_info,16,0,NaN,NaN,NaN,NaN,NaN,NaN
TSD_Label,TSD,NaN,NaN,NaN,NaN,NaN,NaN,RandomForestRegressor,mutual_info,146,0,NaN,0.465726060341605,61.24772854186175,45.4287514910119,0.6854942661039224,0.6632301433042371


## Test on external test set

In [5]:
col2tasks = [
    {"ts_label": "TSD", "ss_label": "SSD"},
    {"water_label": "WS24_water", "water4_label": "WS24_water4", "acid_label": "WS24_acid", "base_label": "WS24_base", "boiling_label": "WS24_boiling"}
]
task2col = {"TSD": "Label", "SSD": "Label"}

task2col.update({t: t.split("_")[-1] + "_label" for t in ["WS24_water", "WS24_water4", "WS24_acid", "WS24_base", "WS24_boiling"]})
split = "external_test"
results = []
model_name = "ML_retrained"
evaluation_dir = log_dir/model_name
evaluation_dir.mkdir(exist_ok=True, parents=True)
for df, col2task in zip([df_ts, df_ws], col2tasks):
    for label_col, task in col2task.items():
        row = df_ml_our[df_ml_our["Task"] == task].iloc[0]
        model_file = f'total_model_{row["feature_selector"]}{row["feature_num"]}_none1_{row["model_name"]}_{row["random_state"]}.model'
        model_path = model_root_dir / task.split("_")[0] / "RAC_and_zeo_features_with_id_prop" / task2col[task] / model_file
        print(model_path)
        if not os.path.exists(model_path):
            print(f"{model_path} does not exist")
            continue
        if task == "TSD":
            model = RegressionModel(random_state=0)
            model.load_total_model(model_path)
            model.train(model.model, model.params)
            y_pred = model.predict(df.loc[:, "Di":]).squeeze()
            y_true = df[label_col].values
            r2, rmse, mae, pearson_corr, spearman_corr = model.cal_metrics(y_true, y_pred)
            results.append({"Task": task, "Model": model_name, "R2": r2, "MAE": mae})
            df_pred = pd.DataFrame([df["MofName"].values, y_true, y_pred, np.abs(y_true - y_pred)], 
                         index=["MofName", "GroundTruth", "Predicted", "Error"]).T
            df_pred.to_csv(evaluation_dir/f"{split}_results_{task}.csv", index=False)
        else:
            model = ClassificationModel(random_state=0, n_class=2)
            model.load_total_model(model_path)
            model.train(model.model, model.params)
            y_pred = model.predict(df.loc[:, "Di":], return_prob=False).squeeze()
            y_prob = model.predict(df.loc[:, "Di":], return_prob=True)
            y_true = df[label_col].values
            if task == "WS24_water4":
                y_true = y_true - 1
            acc, bacc, mcc, auc, f1, ce = model.cal_metrics(y_true, y_pred, y_prob, model.n_class)
            results.append({"Task": task, "Model": model_name, "ACC": acc, "BACC": bacc, "AUROC": auc})
            if y_prob.shape[1] == 2:
                y_prob = y_prob[:, 1]
            df_pred = pd.DataFrame([df["MofName"].values, y_true, y_pred, y_prob.tolist()], 
                         index=["MofName", "GroundTruth", "Predicted", "Prob"]).T
            df_pred.to_csv(evaluation_dir/f"{split}_results_{task}.csv", index=False)
df_ml_res = pd.DataFrame(results)
df_ml_res

ML/models/TSD/RAC_and_zeo_features_with_id_prop/Label/total_model_mutual_info146_none1_RandomForestRegressor_0.model
Train/validation num: 2004/501
**************************************************
All results for k-fold cross validation: 
           tr_R2    tr_RMSE    val_R2  val_RMSE     te_R2    te_RMSE
fold_1  0.778716  41.696976  0.467066  62.60925  0.465726  61.247729
mean    0.778716  41.696976  0.467066  62.60925  0.465726  61.247729
Total run time: 00h:00m:00s
ML/models/SSD/RAC_and_zeo_features_with_id_prop/Label/total_model_mutual_info80_none1_RandomForestClassifier_0.model
Train/validation num: 1394/348
**************************************************
All results for k-fold cross validation: 
        val_ACC  val_BACC  val_MCC  val_AUC  val_F1  te_ACC  te_BACC  te_MCC  \
fold_1   0.7443    0.7195   0.4729   0.8601  0.7237  0.7615   0.7261  0.4917   
mean     0.7443    0.7195   0.4729   0.8601  0.7237  0.7615   0.7261  0.4917   

        te_AUC  te_F1  
fold_1  0.8342  0.

/opt/share/miniconda3/envs/mofmthnn/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/opt/share/miniconda3/envs/mofmthnn/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:445: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


**************************************************
All results for k-fold cross validation: 
        val_ACC  val_BACC  val_MCC  val_AUC  val_F1  te_ACC  te_BACC  te_MCC  \
fold_1   0.7857    0.7857   0.6325   0.9235  0.7754  0.8056   0.8056  0.6198   
mean     0.7857    0.7857   0.6325   0.9235  0.7754  0.8056   0.8056  0.6198   

        te_AUC   te_F1  
fold_1  0.9198  0.8042  
mean    0.9198  0.8042  
Total run time: 00h:00m:00s
Scaled X shape: (46, 190)
Variance filtered X shape: (46, 162)
Selected X shape: (46, 6)
Scaled X shape: (46, 190)
Variance filtered X shape: (46, 162)
Selected X shape: (46, 6)
ML/models/WS24/RAC_and_zeo_features_with_id_prop/base_label/total_model_mutual_info16_none1_GaussianProcessClassifier_0.model
Train/validation num: 80/20
**************************************************
All results for k-fold cross validation: 
        val_ACC  val_BACC  val_MCC  val_AUC  val_F1  te_ACC  te_BACC  te_MCC  \
fold_1      0.7       0.7   0.4082     0.85   0.697  0.708

,Task,Model,R2,MAE,ACC,BACC,AUROC
0,TSD,ML_retrained,0.349546,53.829348,NaN,NaN,NaN
1,SSD,ML_retrained,NaN,NaN,0.7500,0.7600,0.8560
2,WS24_water,ML_retrained,NaN,NaN,0.6087,0.3889,0.5389
3,WS24_water4,ML_retrained,NaN,NaN,0.2826,0.2212,0.6007
4,WS24_acid,ML_retrained,NaN,NaN,0.4130,0.4410,0.4667
5,WS24_base,ML_retrained,NaN,NaN,0.5435,0.5229,0.5200
6,WS24_boiling,ML_retrained,NaN,NaN,0.4783,0.2619,0.2440


In [6]:
df_ml_res["Model"] = "ML_retrained"
df_ml_res.set_index(keys=['Task', 'Model'], inplace=True)
df_ml_res.to_excel(log_dir/"ML_results_external_test.xlsx")

## Get Model hyperparameters

In [7]:
col2tasks = [
    {"ts_label": "TSD", "ss_label": "SSD"},
    {"water_label": "WS24_water", "water4_label": "WS24_water4", "acid_label": "WS24_acid", "base_label": "WS24_base", "boiling_label": "WS24_boiling"}
]
task2col = {"TSD": "Label", "SSD": "Label"}

task2col.update({t: t.split("_")[-1] + "_label" for t in ["WS24_water", "WS24_water4", "WS24_acid", "WS24_base", "WS24_boiling"]})
split = "external_test"
results = []
model_name = "ML_retrained"
evaluation_dir = log_dir/model_name
evaluation_dir.mkdir(exist_ok=True, parents=True)
for df, col2task in zip([df_ts, df_ws], col2tasks):
    for label_col, task in col2task.items():
        row = df_ml_our[df_ml_our["Task"] == task].iloc[0]
        model_file = f'total_model_{row["feature_selector"]}{row["feature_num"]}_none1_{row["model_name"]}_{row["random_state"]}.model'
        model_path = model_root_dir / task.split("_")[0] / "RAC_and_zeo_features_with_id_prop" / task2col[task] / model_file
        print("="*50)
        print("Model:", task + "_" + row["model_name"], "feature_selector:", row["feature_selector"], "feature_num:", row["feature_num"])
        if not os.path.exists(model_path):
            print(f"{model_path} does not exist")
            continue
        if task == "TSD":
            model = RegressionModel(random_state=0)
            model.load_total_model(model_path)
            print("model params:", model.params)
            
        else:
            model = ClassificationModel(random_state=0, n_class=2)
            model.load_total_model(model_path)
            print("model params:", model.params)

Model: TSD_RandomForestRegressor feature_selector: mutual_info feature_num: 146
model params: {'max_leaf_nodes': 135, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 116}
Model: SSD_RandomForestClassifier feature_selector: mutual_info feature_num: 80
model params: {'class_weight': 'balanced_subsample', 'max_leaf_nodes': 146, 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 462}
Model: WS24_water_RandomForestClassifier feature_selector: RFE feature_num: 85
model params: {'class_weight': 'balanced_subsample', 'max_leaf_nodes': 119, 'min_samples_leaf': 4, 'min_samples_split': 11, 'n_estimators': 51}
Model: WS24_water4_RandomForestClassifier feature_selector: mutual_info feature_num: 171
model params: {'class_weight': 'balanced', 'max_leaf_nodes': 53, 'min_samples_leaf': 1, 'min_samples_split': 16, 'n_estimators': 121}
Model: WS24_acid_GaussianProcessClassifier feature_selector: RFE feature_num: 6
model params: {'kernel': 7.61**2 * RBF(length_scale=38.2) + Whi